# GemmaCare0625-LoRA: LoRA Fine-Tuning of Gemma-2B

**Author:** Olisemeka Nmarkwe  

This notebook fine-tunes Google's Gemma-2B model using LoRA (Low-Rank Adaptation) for efficient parameter updates.

## 1. Install Dependencies

Install all required libraries for the fine-tuning process.

In [ ]:
!pip install --quiet transformers datasets accelerate peft huggingface_hub

## 2. Imports and Authentication

Import necessary libraries and authenticate with Hugging Face Hub.

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import pandas as pd

# Log in to Hugging Face (interactive prompt)
login()

## 3. Configuration

Set up all training parameters and model configuration.

In [ ]:
BASE_MODEL = "google/gemma-2b"
MODEL_NAME = "GemmaCare0625-LoRA"
HUB_REPO   = "OliseNS/GemmaCare0625-LoRA"
EPOCHS     = 10
BATCH_SIZE = 4
LEARNING_RATE = 2e-4
MAX_LENGTH = 1024

print(f"Training Configuration:")
print(f"Base Model: {BASE_MODEL}")
print(f"Model Name: {MODEL_NAME}")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Max Length: {MAX_LENGTH}")

## 4. Upload Data Files

Upload your training and validation data files. Uncomment the appropriate section based on your platform:
- **Colab**: Use the files.upload() method
- **Kaggle**: Use the Kaggle dataset upload interface
- **Local**: Ensure files are in the correct directory

In [ ]:
# For Google Colab - uncomment the lines below:
# from google.colab import files
# uploaded = files.upload()  # select train.jsonl and validation.jsonl

# For Kaggle or local environment, ensure files are in the working directory
train_file = "train.jsonl"
val_file   = "validation.jsonl"

# Verify files exist
import os
print(f"Training file exists: {os.path.exists(train_file)}")
print(f"Validation file exists: {os.path.exists(val_file)}")

## 5. Quick Data Inspection

Examine the structure and content of the training data.

In [ ]:
print("Inspecting training data..."))
df = pd.read_json(train_file, lines=True)
print(f"Rows: {len(df)}, Columns: {df.columns.tolist()}")
print("\nFirst example:")
print(df.head(1).to_dict(orient='records')[0])

print("\nValidation data:")
df_val = pd.read_json(val_file, lines=True)
print(f"Validation rows: {len(df_val)}")

## 6. Load Dataset

Load the training and validation datasets using Hugging Face datasets library.

In [ ]:
dataset = load_dataset(
    "json",
    data_files={"train": train_file, "validation": val_file}
)

print(f"Dataset loaded successfully!")
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")

## 7. Initialize Tokenizer and Model

Load the base Gemma-2B model and tokenizer.

In [ ]:
print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
    device_map='auto'
)

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded with {model.num_parameters():,} parameters")

## 8. Configure and Attach LoRA Adapter

Set up LoRA configuration for efficient fine-tuning.

In [ ]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05
)

model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

## 9. Data Preprocessing

Tokenize the dataset for training.

In [ ]:
def tokenize_fn(examples):
    combined = [p + c for p, c in zip(examples['prompt'], examples['completion'])]
    tokens = tokenizer(
        combined,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens

print("Tokenizing datasets...")
tokenized = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=['prompt', 'completion']
)

print("Tokenization complete!")

## 10. Setup Training Arguments and Trainer

Configure the training parameters and initialize the trainer.

In [ ]:
training_args = TrainingArguments(
    output_dir='checkpoints',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    logging_steps=50,
    learning_rate=LEARNING_RATE,
    fp16=True,
    push_to_hub=True,
    hub_model_id=HUB_REPO
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer
)

print("Trainer initialized successfully!")

## 11. Start Training

Begin the 10-epoch training process.

In [ ]:
print(f"Starting training of {MODEL_NAME}..."))
print(f"Training for {EPOCHS} epochs with batch size {BATCH_SIZE}")

# Start training
trainer.train()

print("Training completed!")

## 12. Evaluate and Save Model

Evaluate the final model and save/upload to Hugging Face Hub.

In [ ]:
# Final evaluation
print("Performing final evaluation...")
metrics = trainer.evaluate()
print("Final metrics:")
for key, value in metrics.items():
    print(f"{key}: {value}")

# Push to Hub
print(f"\nUploading {MODEL_NAME} to Hugging Face Hub...")
trainer.push_to_hub(commit_message=f"Final {MODEL_NAME} model")

# Save locally
print(f"\nSaving {MODEL_NAME} adapter locally...")
model.save_pretrained(f'{MODEL_NAME}-adapter')
tokenizer.save_pretrained(f'{MODEL_NAME}-adapter')

print(f"\n✅ All done! Your {MODEL_NAME} adapter is ready for use or sharing via Hugging Face.")
print(f"🔗 Model available at: https://huggingface.co/{HUB_REPO}")

## 13. Model Testing (Optional)

Test the fine-tuned model with a sample prompt.

In [ ]:
# Optional: Test the model
test_prompt = "What are the symptoms of diabetes?"
print(f"Testing model with prompt: '{test_prompt}'")

inputs = tokenizer(test_prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nModel response: {response}")

---

## Summary

This notebook successfully fine-tuned the Gemma-2B model using LoRA for the GemmaCare0625 project. The model has been trained for 10 epochs and is now available for inference or further fine-tuning.

**Key Features:**
- ✅ LoRA fine-tuning for efficient parameter updates
- ✅ 10-epoch training with evaluation
- ✅ Automatic model upload to Hugging Face Hub
- ✅ Compatible with Kaggle, Colab, and other platforms

**Author:** Olisemeka Nmarkwe  
**Model Repository:** [OliseNS/GemmaCare0625-LoRA](https://huggingface.co/OliseNS/GemmaCare0625-LoRA)